Для временного ряда bike-sharing построить три модели:
* Датасет аггрегировать по неделям
* Простое экспоненциальное сглаживание
* Двойное экспоненциальное сглаживание
* Тройное экспоненциальное сглаживание (тип модели additive или multiplicative) выбрать самостоятельно 


Для финального отчета сделать кросс-валидацию, пояснить выбранные метрики, а так же их финальные значения. 





In [377]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from plotly.graph_objects import *
from plotly.offline import init_notebook_mode,iplot
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing  

df = pd.read_csv('../datatest/bike-sharing.csv')
df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,9.84,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,9.02,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,9.02,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,9.84,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,9.84,0.2879,0.75,0.0,0,1,1


In [378]:
df = df[['dteday','cnt']]
df['dteday'] = pd.to_datetime(df['dteday'])

# Агрегация по неделям

In [379]:
df.set_index('dteday',inplace=True)
df_agg = df.resample('W').mean()
df_agg.rename(columns = {'cnt' : 'mean'}, inplace = True) # Агрегируем по сумме

df_agg.head()

,mean
dteday,
2011-01-02,38.000000
2011-01-09,58.074074
2011-01-16,55.368098
2011-01-23,57.320261
2011-01-30,55.388489


In [381]:
len_df = len(df_agg.index)-1

# Простое экспоненциальное сглаживание

In [390]:
df_agg['ses'] = SimpleExpSmoothing(df_agg['mean']).fit(smoothing_level = 1/(2*12),
                                                       optimized = False,use_brute = True).fittedvalues

In [391]:
iplot(Figure(data=[Scatter(x = df_agg.index,
                          y = df_agg['mean'],
                          name = 'Original'),
                  Scatter(x = df_agg.index,
                         y = df_agg['ses'],
                         name = 'Simple Exponential Smoothing')]))

# Двойное экспоненциальное сглаживание

In [405]:
df_agg['des_mul']=ExponentialSmoothing(df_agg['mean'], trend='mul').fit().fittedvalues


iplot(Figure(data=[Scatter(x =df_agg.index,
                          y = df_agg['mean'],
                          name = 'Original'),
                  Scatter(x = df_agg.index,
                         y = df_agg['des_mul'],
                         name = 'Double Exponential Smoothing (Multiplicative)')]))

# Тройное экспоненциальное сглаживание

In [406]:
df_agg['tes_mul'] = ExponentialSmoothing(df_agg['mean'], 
                                         trend = 'mul', 
                                         seasonal = 'mul', 
                                         seasonal_periods = 12).fit().fittedvalues

iplot(Figure(data = [Scatter(x =df_agg.index,
                          y = df_agg['mean'],
                          name = 'Original'),
                 
                  Scatter(x = df_agg.index,
                         y = df_agg['tes_mul'],
                         name = 'Tripple Exponential Smoothing (Multiplicative)')]))

In [407]:
train,test = df_agg[0:-50],df_agg[-50:]
mod = ExponentialSmoothing(train['mean'], 
                           trend = 'mul', 
                           seasonal_periods = 48, 
                           seasonal = 'mul' 
                           ).fit(optimized = True)

In [408]:
mod.summary()

Dep. Variable:,mean,No. Observations:,56
Model:,ExponentialSmoothing,SSE,1798.458
Optimized:,True,AIC,298.283
Trend:,Multiplicative,BIC,403.601
Seasonal:,Multiplicative,AICC,6238.283
Seasonal Periods:,48,Date:,"Mon, 13 Dec 2021"
Box-Cox:,False,Time:,01:58:13
Box-Cox Coeff.:,None,,
,coeff,code,optimized
smoothing_level,0.5989615,alpha,True
smoothing_trend,3.8849e-14,beta,True


In [409]:
preds = mod.forecast(len(test))


In [410]:
trace1 = Scatter(x = test.index, y = test['mean'], name = 'Original')
trace2 = Scatter(x = test.index,y = preds, name = 'Exponential Smoothing')
iplot(Figure(data = [trace1, trace2]))